In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

/data0/matthew/Software/NCSA/HAL_Projects/tf_estimator_tests/estimator_test_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#gpus = tf.config.list_physical_devices('GPU')
#if len(gpus) > 1:
#    tf.config.set_visible_devices(gpus[1], 'GPU')

# Prepare input data

In [3]:
def input_fn(mode, input_context=None):
    datasets, info = tfds.load(
        name='mnist',
        with_info=True,
        as_supervised=True)
    mnist_dataset = \
        (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else datasets['test'])
    num_examples = \
        (info.splits['train'].num_examples if mode == tf.estimator.ModeKeys.TRAIN else info.splits['test'].num_examples)

    def normalize_img(image, label):
        return tf.cast(image, tf.float32) / 255., label

    mnist_dataset = mnist_dataset.map(
        normalize_img,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )

    if input_context:
        mnist_dataset = mnist_dataset.shard(
            input_context.num_input_pipelines,
            input_context.input_pipeline_id)

    batch_size = 32
    if mode == tf.estimator.ModeKeys.TRAIN:
        return mnist_dataset.cache().shuffle(num_examples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    else:
        return mnist_dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Estimator from keras

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

2022-08-10 13:25:52.970470: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7369 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1


In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [6]:
session_config = tf.compat.v1.ConfigProto()
# restrict the estimator to running on certain gpus.
session_config.gpu_options.visible_device_list = "1"
estimator_config = tf.estimator.RunConfig(
    model_dir = "./mnist_estimator_mdl_test",
    session_config=session_config)

In [7]:
est_model = tf.keras.estimator.model_to_estimator(
    keras_model=model)
    #config=estimator_config)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


/data0/matthew/Software/NCSA/HAL_Projects/tf_estimator_tests/estimator_test_venv/lib/python3.8/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-08-10 13:26:03.004647: E tensorflow/core/common_runtime/session.cc:91] Failed to create session: ALREADY_EXISTS: TensorFlow device (GPU:0) is being mapped to multiple devices (0 now, and 1 previously), which is not supported. This may be the result of providing different GPU configurations (ConfigProto.gpu_options, for example different visible_device_list) when creating multiple Sessions in the same process. This is not currently supported, see https://github.com/tensorflow/tensorflow/issues/19083
2022-08-10 13:26:03.004667: E tensorflow/c/c_api.cc:2267] ALR

AlreadyExistsError: TensorFlow device (GPU:0) is being mapped to multiple devices (0 now, and 1 previously), which is not supported. This may be the result of providing different GPU configurations (ConfigProto.gpu_options, for example different visible_device_list) when creating multiple Sessions in the same process. This is not currently supported, see https://github.com/tensorflow/tensorflow/issues/19083

In [15]:
# estimator train works on 'steps' not epochs.
est_mdl = est_model.train(input_fn=input_fn, steps=20000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Saving checkpoints for 2975 into /tmp/tmp8h1tzipq/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 2975...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 2975...


INFO:tensorflow:Loss for final step: 0.18070923.


INFO:tensorflow:Loss for final step: 0.18070923.


# Estimator using custom model_fn

In [2]:
def input_fn(mode, input_context=None):
    datasets, info = tfds.load(
        name='mnist',
        with_info=True,
        as_supervised=True)
    mnist_dataset = \
        (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else datasets['test'])
    num_examples = \
        (info.splits['train'].num_examples if mode == tf.estimator.ModeKeys.TRAIN else info.splits['test'].num_examples)

    def normalize_img(image, label):
        return tf.cast(image, tf.float32) / 255., label

    mnist_dataset = mnist_dataset.map(
        normalize_img,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )

    if input_context:
        mnist_dataset = mnist_dataset.shard(
            input_context.num_input_pipelines,
            input_context.input_pipeline_id)

    batch_size = 32
    if mode == tf.estimator.ModeKeys.TRAIN:
        return mnist_dataset.cache().shuffle(num_examples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    else:
        return mnist_dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [3]:
def model_fn(features, labels, mode):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='sigmoid')
    ])

    if mode == tf.estimator.ModeKeys.TRAIN:
        preds = model(features, training=True)
        optimizer = tf.compat.v1.train.AdamOptimizer()
        loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False)(labels, preds)
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op = optimizer.minimize(
                loss, tf.compat.v1.train.get_or_create_global_step())
        )
    elif mode == tf.estimator.ModeKeys.PREDICT:
        preds = model(features, training=True)
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=preds
        )
    elif mode == tf.estimator.ModeKeys.EVAL:
        preds = model(features, training=True)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False)(labels, preds)
        acc = tf.compat.v1.metrics.accuracy(labels, tf.argmax(preds, axis=-1))
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            eval_metric_ops={'acc': acc}
        )

In [4]:
session_config = tf.compat.v1.ConfigProto()
# restrict the estimator to running on certain gpus.
session_config.gpu_options.visible_device_list = "1"
estimator_config = tf.estimator.RunConfig(
    model_dir = "./mnist_estimator_mdl_test",
    session_config=session_config)

In [5]:
mdl_est = tf.estimator.Estimator(
    model_fn=model_fn,
    config=estimator_config
)

INFO:tensorflow:Using config: {'_model_dir': './mnist_estimator_mdl_test', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  visible_device_list: "1"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
mdl_est.train(input_fn=input_fn, steps=20000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 9375...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 9375...


INFO:tensorflow:Loss for final step: 0.04477999.


INFO:tensorflow:Loss for final step: 0.04477999.


In [44]:
mdl_est.evaluate(input_fn=input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-04-19T11:03:49


...

INFO:tensorflow:Saving dict for global step 7500: acc = 0.9747, global_step = 7500, loss = 0.07828136


INFO:tensorflow:Saving dict for global step 7500: acc = 0.9747, global_step = 7500, loss = 0.07828136


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7500: ./mnist_estimator_mdl_test/model.ckpt-7500


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7500: ./mnist_estimator_mdl_test/model.ckpt-7500


{'acc': 0.9747, 'loss': 0.07828136, 'global_step': 7500}

# Estimator using custom model_fn and trying to use V1 only

In [2]:
def input_fn(mode, input_context=None):
    datasets, info = tfds.load(
        name='mnist',
        with_info=True,
        as_supervised=True)
    mnist_dataset = \
        (datasets['train'] if mode == tf.compat.v1.estimator.ModeKeys.TRAIN else datasets['test'])
    num_examples = \
        (info.splits['train'].num_examples if mode == tf.compat.v1.estimator.ModeKeys.TRAIN else info.splits['test'].num_examples)

    def normalize_img(image, label):
        return tf.cast(image, tf.float32) / 255., label

    mnist_dataset = mnist_dataset.map(
        normalize_img,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )

    if input_context:
        mnist_dataset = mnist_dataset.shard(
            input_context.num_input_pipelines,
            input_context.input_pipeline_id)

    batch_size = 32
    if mode == tf.compat.v1.estimator.ModeKeys.TRAIN:
        return mnist_dataset.cache().shuffle(num_examples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    else:
        return mnist_dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [45]:
def model_fn(features, labels, mode):
    model = tf.compat.v1.keras.models.Sequential([
        tf.compat.v1.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.compat.v1.keras.layers.Dense(128, activation='relu'),
        tf.compat.v1.keras.layers.Dense(10, activation='sigmoid')
    ])

    if mode == tf.compat.v1.estimator.ModeKeys.TRAIN:
        preds = model(features, training=True)
        optimizer = tf.compat.v1.train.AdamOptimizer()
        loss = tf.compat.v1.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False)(labels, preds)
        return tf.compat.v1.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op = optimizer.minimize(
                loss, tf.compat.v1.train.get_or_create_global_step())
        )
    elif mode == tf.compat.v1.estimator.ModeKeys.PREDICT:
        preds = model(features, training=True)
        return tf.compat.v1.estimator.EstimatorSpec(
            mode=mode,
            predictions=preds
        )
    elif mode == tf.compat.v1.estimator.ModeKeys.EVAL:
        preds = model(features, training=True)
        loss = tf.compat.v1.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False)(labels, preds)
        acc = tf.compat.v1.metrics.accuracy(labels, tf.compat.v1.argmax(preds, axis=-1))
        return tf.compat.v1.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            eval_metric_ops={'acc': acc}
        )

In [47]:
session_config = tf.compat.v1.ConfigProto()
# restrict the estimator to running on certain gpus.
session_config.gpu_options.visible_device_list = "1"
estimator_config = tf.compat.v1.estimator.RunConfig(
    model_dir = "./mnist_estimator_mdl_test_v1",
    session_config=session_config)

In [52]:
mdl_est = tf.compat.v1.estimator.Estimator(
    model_fn=model_fn,
    config=estimator_config
)

INFO:tensorflow:Using config: {'_model_dir': './mnist_estimator_mdl_test_v1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  visible_device_list: "1"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './mnist_estimator_mdl_test_v1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  visible_device_list: "1"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [53]:
mdl_est.train(input_fn=input_fn, steps=20000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Saving checkpoints for 3750 into ./mnist_estimator_mdl_test_v1/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 3750...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 3750...


INFO:tensorflow:Loss for final step: 0.12676439.


INFO:tensorflow:Loss for final step: 0.12676439.


In [54]:
mdl_est.evaluate(input_fn=input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-04-19T11:09:12


...

INFO:tensorflow:Saving dict for global step 3750: acc = 0.9711, global_step = 3750, loss = 0.09722594


INFO:tensorflow:Saving dict for global step 3750: acc = 0.9711, global_step = 3750, loss = 0.09722594


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3750: ./mnist_estimator_mdl_test_v1/model.ckpt-3750


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3750: ./mnist_estimator_mdl_test_v1/model.ckpt-3750


{'acc': 0.9711, 'loss': 0.09722594, 'global_step': 3750}

In [55]:
type(mdl_est)

tensorflow_estimator.python.estimator.estimator.Estimator